In [ ]:
pip install requests beautifulsoup4 requests-html

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.4/83.4 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 10.4 MB/s eta 0:00:00
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1257 sha256=b9d2e47ab2fc1ceca453853028c7d4202d71cc4300a883ef29272ebe73169f94
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


In [ ]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 46.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 95.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.3 MB/s eta 0:00:00


In [ ]:
pip install tensorflow==2.12.

We will use this popular and effective model:
https://huggingface.co/jonaskoenig/topic_classification_04?text=learning+is+good

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("jonaskoenig/topic_classification_04")
model = AutoModelForSequenceClassification.from_pretrained("jonaskoenig/topic_classification_04", from_tf=True)

All TF 2.0 model weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForSequenceClassification for predictions without further training.


In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="jonaskoenig/topic_classification_04")

Some layers from the model checkpoint at jonaskoenig/topic_classification_04 were not used when initializing TFBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at jonaskoenig/topic_classification_04.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [ ]:
text = '''A man accused of stealing thousands of dollars in cryptocurrency from an ex-girlfriend "likely" did it, according to a B.C. judge, but the challenges of tracking those virtual coins mean he has avoided a conviction.

Though Orion Alexander Holtby was found not guilty of fraud and theft, B.C. Supreme Court Justice Ward Branch urged him to find a way to help the alleged victim reclaim her digital investment, valued at about $92,000 in March 2021.'''
#text = 'The Toronto Raptors need new players'
inputs = tokenizer.encode_plus(text, padding="longest", truncation=True, return_tensors="pt")

In [ ]:
outputs = model(**inputs)
logits = outputs.logits
logits

tensor([[ 1.4518,  3.0202, -2.3974,  0.3862,  1.4785, -4.2111,  0.4622, -0.9270,
         -1.6790,  1.7816]], grad_fn=<AddmmBackward0>)

{0: 'society and culture',
1:'Science',
2:'Health',
3:'education',
4:'Computers and Internet',
5:'sports',
6:'Finance',
7: 'Entertainment and Music',
8:'Family and Relationships',
9:'Politics and Government'}

In [ ]:
probabilities = logits.softmax(dim=1)
#list((probabilities[0])).index(max(list((probabilities[0]))))
probabilities = list(probabilities[0])
for i in range(0,len(probabilities)):
  print(probabilities[i].item())

0.10995951294898987
0.5277031064033508
0.002341808984056115
0.03788508102297783
0.11293784528970718
0.000381833961000666
0.04087519645690918
0.010189695283770561
0.004803686868399382
0.15292227268218994


In [ ]:
sorted=(list(probabilities[0])).sort()


None


In [ ]:
def classify_text(some_text):
  categories = {0: 'society and culture',
1:'Science',
2:'Health',
3:'education',
4:'Computers and Internet',
5:'sports',
6:'Finance',
7: 'Entertainment and Music',
8:'Family and Relationships',
9:'Politics and Government'}
  inputs = tokenizer.encode_plus(some_text, padding="longest", truncation=True, return_tensors="pt")
  outputs = model(**inputs)
  logits = outputs.logits
  probabilities = logits.softmax(dim=1)
  if max(list((probabilities[0]))) > .6:
    index = list((probabilities[0])).index(max(list((probabilities[0]))))
    return categories[index]
  else:
    probabilities = list(probabilities[0])
    print((probabilities))
    prob_list = []
    categories_list = []
    for i in range(0,len(probabilities)):
      prob_list.append(probabilities[i].item())
    prob_list = prob_list.sort()
    for i in range(0,len(probabilities)):
        if probabilities[i].item() > .1:
          categories_list.append(categories[probabilities[i].item()])
          #print(categories[i])

    return categories_list


In [ ]:
#text = "it's important to eat healthy"
print(classify_text(text))

[tensor(0.1100, grad_fn=<UnbindBackward0>), tensor(0.5277, grad_fn=<UnbindBackward0>), tensor(0.0023, grad_fn=<UnbindBackward0>), tensor(0.0379, grad_fn=<UnbindBackward0>), tensor(0.1129, grad_fn=<UnbindBackward0>), tensor(0.0004, grad_fn=<UnbindBackward0>), tensor(0.0409, grad_fn=<UnbindBackward0>), tensor(0.0102, grad_fn=<UnbindBackward0>), tensor(0.0048, grad_fn=<UnbindBackward0>), tensor(0.1529, grad_fn=<UnbindBackward0>)]


KeyError: ignored

In [ ]:
text = "This is an example sentence."
inputs = tokenizer.encode_plus(text, padding="longest", truncation=True, return_tensors="pt")

In [ ]:
outputs = model(**inputs)
logits = outputs.logits

<function SequenceClassifierOutput.fromkeys(iterable, value=None)>

NLTK is a good library with simple yet very effective functions
https://www.nltk.org/

In [ ]:
import pandas as pd
from urllib.parse import urlparse
import numpy as np
import nltk.data
from requests_html import HTMLSession

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
def parse_elements(url,element):
    session = HTMLSession()
    r = session.get(url)
    elements = r.html.find(element)
    return elements

In [ ]:
def sentiment_detection(sentence):
    positive_words = ('happy', 'sunny', 'positive', 'triumphant', 'optimistic', 'wonderful', 'significant', 'achieve','capabilities','progress','prized')
    negative_words = ('sad', 'terrible', 'frightening', 'rainy', 'scary', 'shocked', 'critical','bottlenecked')

    positive = any(sentence.count(i) > 0 for i in positive_words)
    negative = any(sentence.count(i) > 0 for i in negative_words)

    if positive == negative == False:
        return "neutral"
    elif positive != negative:
        return "positive" if positive else "negative"
    else:
        return "mixed"

In [ ]:
def topic_detection(sentence):
    DataScience_words = ('data science', 'regression', 'predict', 'predictive', 'classify')
    AI_words = ('AI', 'machine learning', 'ml', 'ML', 'NLP', 'LLM','deep learning','neural network','generative', 'GPT-3')
    DataWords = ('analysis', 'algorithms','data','data structure','data-structure','big data')
    DataScience = any(sentence.count(i) > 0 for i in DataScience_words)
    AI = any(sentence.count(i) > 0 for i in AI_words)
    data = any(sentence.count(i) > 0 for i in DataWords)
    topics = []
    if DataScience == True:
        topics.append("Data Science")
    if AI == True:
        topics.append("AI")
    if data == True:
        topics.append("data")
    return topics

In [ ]:
def sentence_list(paragraph_string):
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    list_of_sentences = tokenizer.tokenize(paragraph_string)
    return list_of_sentences

In [ ]:
paragraph='''"We have this weird relationship with alcohol where it's part of our lives, but as soon as it goes into public we look at it as potentially problematic," said Dan Malleck, the chair of Health Sciences at Brock University in St. Catharines, Ont., and a medical historian specializing in drug and alcohol regulation and policy.

"In my view, a lot of the general associations we have with drinking in public are negative, like drunkenness in public, drinking and driving, like drunken hoodlums, all of these things — which make the news, but aren't necessarily the only way people consume alcohol in public."'''



In [ ]:
sentences = sentence_list(paragraph)
print(sentences)
#for sentence in sentences:
#  print(sentence)

['"We have this weird relationship with alcohol where it\'s part of our lives, but as soon as it goes into public we look at it as potentially problematic," said Dan Malleck, the chair of Health Sciences at Brock University in St. Catharines, Ont., and a medical historian specializing in drug and alcohol regulation and policy.', '"In my view, a lot of the general associations we have with drinking in public are negative, like drunkenness in public, drinking and driving, like drunken hoodlums, all of these things — which make the news, but aren\'t necessarily the only way people consume alcohol in public."']
